In [200]:
import requests
import pickle


# Set up the API call variables to get season data
game_data = [] # Empty, to be filled
year = '2010' # Season of interest
season_type = '02' # pre/regular/playoff (01/02/03)
max_game_ID = 1290 # Max number of games in a year. 

In [201]:
# Make the call to the API, get the data and save the data
# Note this is not a trivial cell. Load it only when necessary
for i in range(1, 1290):
    r = requests.get(url='http://statsapi.web.nhl.com/api/v1/game/'
        + year + season_type +str(i).zfill(4)+'/feed/live')
    data = r.json()
    game_data.append(data)  

In [202]:
# I guess for now we'll save it as a pkl even though I don't understand why


with open('./'+year+'Test_Dataset.pkl', 'wb') as f:
    pickle.dump(game_data, f, pickle.HIGHEST_PROTOCOL)

In [203]:
# Follow along with this guys stuff to learn how data is stored in the API

import numpy as np 
import pandas as pd 

with open('2010Test_Dataset.pkl', 'rb') as f:
    game_data = pickle.load(f)

In [90]:
# For this purpose we are interested in shots and goals. Make these event types
# Make a dictionary also so that we can keep track of shot coordinates
event_types = ['Shots', 'Goals']

league_data = {}
league_data['Shots'] = {}
league_data['Shots']['x'] = []
league_data['Shots']['y'] = []

league_data['Goals'] = {}
league_data['Goals']['x'] = []
league_data['Goals']['y'] = []

league_data

{'Shots': {'x': [], 'y': []}, 'Goals': {'x': [], 'y': []}}

In [91]:
for data in game_data:
    if 'liveData' not in data: # This part checks teh data to be sure that it has what we want!
        continue 
    plays = data['liveData']['plays']['allPlays']   #Get all the plays
    for play in plays: #check each play
        for event in event_types: # cycle through the desired events
            if play['result']['event'] in [event]: # Check if the play matches the event
                if 'x' in play['coordinates']: # Check that coordinates actually exist
                    
                    #Save it all
                    league_data['event']['x'].append(play['coordinates']['x'])
                    league_data['event']['y'].append(play['coordinates']['y'])

In [204]:

games = []
for data in game_data:
    if 'liveData' not in data:
        continue
    home = data['liveData']['linescore']['teams']['home']['team']['abbreviation']
    home_goals = data['liveData']['linescore']['teams']['home']['goals']
    home_shots = data['liveData']['linescore']['teams']['home']['shotsOnGoal']
    away = data['liveData']['linescore']['teams']['away']['team']['abbreviation']
    away_goals = data['liveData']['linescore']['teams']['away']['goals']
    away_shots = data['liveData']['linescore']['teams']['away']['shotsOnGoal']
    games.append([home, away, home_shots, away_shots, home_goals, away_goals])        
    #for i in data['liveData']['linescore']['teams']['home']['shotsOnGoal']:
    #    print(i)

In [205]:
# Save data as .csv
col = ['home', 'away', 'home shots', 'away shots', 'home goals', 'away goals']
df = pd.DataFrame(games)
df.columns = col
df.to_csv("data/NHL_20102011.csv", sep=',',index=False)


In [1]:
import numpy as np

mean = np.mean([14.5, 15.5, 16.5, 14.5, 13.5, 13.25, 14.25, 16.5, 14])
std = np.std([14.5, 15.5, 16.5, 14.5, 13.5, 13.25, 14.25, 16.5, 14])
print(mean, std)

14.722222222222221 1.1269701541722663
